# Homework 4: Deployment

## By: Lorenzo Cesconetto

Check out all the questions [here](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2025/04-deployment/homework.md).


In [31]:
import pandas as pd
import pickle
import numpy as np
import os

## Q1. Notebook

We'll start with the same notebook we ended up with in homework 1.
We cleaned it a little bit and kept only the scoring part. You can find the initial notebook [here](homework/starter.ipynb).

Run this notebook for the March 2023 data.

What's the standard deviation of the predicted duration for this dataset?

- 1.24
- 6.24
- 12.28
- 18.28


In [7]:
with open("model.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

/Users/lorenzocesconetto/Developer/Tutorials/zoomcamp-mlops-2025/.venv/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/lorenzocesconetto/Developer/Tutorials/zoomcamp-mlops-2025/.venv/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
categorical = ["PULocationID", "DOLocationID"]


def read_data(year: int = 2023, month: int = 3, type: str = "yellow"):
    df = pd.read_parquet(
        f"https://d37ci6vzurychx.cloudfront.net/trip-data/{type}_tripdata_{year:04d}-{month:02d}.parquet"
    )

    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype("int").astype("str")

    return df

In [ ]:
year = 2023
month = 3
df = read_data(year, month, "yellow")

In [17]:
dicts = df[categorical].to_dict(orient="records")
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [21]:
print(f"Standard deviation of duration predictions: {np.std(y_pred):.3f}")

Standard deviation of duration predictions: 6.247


## Q2. Preparing the output

Like in the course videos, we want to prepare the dataframe with the output.

First, let's create an artificial `ride_id` column:

```python
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results.

Save it as parquet:

```python
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```

What's the size of the output file?

- 36M
- 46M
- 56M
- 66M

**Note:** Make sure you use the snippet above for saving the file. It should contain only these two columns. For this question, don't change the
dtypes of the columns and use `pyarrow`, not `fastparquet`.


In [ ]:
df["prediction"] = y_pred

In [23]:
df["ride_id"] = f"{year:04d}/{month:02d}_" + df.index.astype("str")

In [30]:
file_name = "tmp_data.parquet"
df[["ride_id", "prediction"]].to_parquet(
    file_name,
    engine="pyarrow",
    compression=None,
    index=False,
)

In [36]:
print(f"File size: {os.path.getsize(file_name) / 1024 ** 2:.1f} Mb")

File size: 65.5 Mb


## Q3. Creating the scoring script

Now let's turn the notebook into a script.

Which command you need to execute for that?


In [38]:
!jupyter nbconvert --to script homework.ipynb
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook homework.ipynb to script
[NbConvertApp] Writing 3537 bytes to homework.py


## Q4. Virtual environment

Now let's put everything into a virtual environment. We'll use pipenv for that.

Install all the required libraries. Pay attention to the Scikit-Learn version: it should be the same as in the starter
notebook.

After installing the libraries, pipenv creates two files: `Pipfile`
and `Pipfile.lock`. The `Pipfile.lock` file keeps the hashes of the
dependencies we use for the virtual env.

What's the first hash for the Scikit-Learn dependency?


In [39]:
print(
    "Scikit-Learn first hash: sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c"
)

Scikit-Learn first hash: sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c


## Q5. Parametrize the script

Let's now make the script configurable via CLI. We'll create two
parameters: year and month.

Run the script for April 2023.

What's the mean predicted duration?

- 7.29
- 14.29
- 21.29
- 28.29

Hint: just add a print statement to your script.


In [40]:
!python homework.py --year 2023 --month 4

Standard deviation of duration predictions: 6.354


## Q6. Docker container

Finally, we'll package the script in the docker container.
For that, you'll need to use a base image that we prepared.

This is what the content of this image is:

```dockerfile
FROM python:3.10.13-slim

WORKDIR /app
COPY [ "model2.bin", "model.bin" ]
```

Note: you don't need to run it. We have already done it.

It is pushed to [`agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim`](https://hub.docker.com/layers/agrigorev/zoomcamp-model/mlops-2024-3.10.13-slim/images/sha256-f54535b73a8c3ef91967d5588de57d4e251b22addcbbfb6e71304a91c1c7027f?context=repo),
which you need to use as your base image.

That is, your Dockerfile should start with:

```dockerfile
FROM agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

# do stuff here
```

This image already has a pickle file with a dictionary vectorizer
and a model. You will need to use them.

Important: don't copy the model to the docker image. You will need
to use the pickle file already in the image.

Now run the script with docker. What's the mean predicted duration
for May 2023?

- 0.19
- 7.24
- 14.24
- 21.19


In [42]:
print("Mean predicted duration: 0.192")

Mean predicted duration: 0.192


## Bonus: upload the result to the cloud (Not graded)

Just printing the mean duration inside the docker image
doesn't seem very practical. Typically, after creating the output
file, we upload it to the cloud storage.

Modify your code to upload the parquet file to S3/GCS/etc.

## Bonus: Use an orchestrator for batch inference

Here we didn't use any orchestration. In practice we usually do.

- Split the code into logical code blocks
- Use a workflow orchestrator for the code execution

## Publishing the image to dockerhub

This is how we published the image to Docker hub:

```bash
docker build -t mlops-zoomcamp-model:2024-3.10.13-slim .
docker tag mlops-zoomcamp-model:2024-3.10.13-slim agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

docker login --username USERNAME
docker push agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim
```

This is just for your reference, you don't need to do it.

## Submit the results

- Submit your results here: https://courses.datatalks.club/mlops-zoomcamp-2025/homework/hw4
- It's possible that your answers won't match exactly. If it's the case, select the closest one.
